In [158]:
import numpy as np
from enum import Enum


In [159]:
# Possible states 0(Sunny), 1(Cloudy), 2(Rainy)
class Weather(Enum):
    SUNNY = 0
    CLOUDY = 1
    RAINY = 2

class WeatherMarkovChain:
    def __init__(self):
        self.transition_matrix = np.array([
        [0.8, 0.2, 0],
        [0.4, 0.4, 0.2],
        [0.2, 0.6, 0.2]
    ])

    def step(self, current_state, current_bel):
        next_state = np.random.choice(
        [Weather.SUNNY.value, Weather.CLOUDY.value, Weather.RAINY.value],
        p = self.transition_matrix[current_state])

        # Compute next state belief.
        next_bel = np.dot(transition_matrix.T, current_bel)

        return next_state, next_bel

    def simulate(self, steps):
        # Initialize current state and belief.
        current_state = Weather.SUNNY.value

        # Current bel is 100% sunny.
        current_bel = np.array([[0.333],
                        [0.333],
                        [0.3333]])

        for _ in range(steps):
            next_state, next_bel = self.step(current_state, current_bel)
            current_state, current_bel = next_state, next_bel

        return current_state, current_bel

# State transition matrix for the Markov Chain
transition_matrix = np.array([
    [0.8, 0.2, 0],
    [0.4, 0.4, 0.2],
    [0.2, 0.6, 0.2]
])

In [160]:
weather_simulator = WeatherMarkovChain()

current_state, current_bel = weather_simulator.simulate(1000)
print("Final belief \n", current_bel)

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(transition_matrix.T)

# Find the eigenvector corresponding to eigenvalue 1
# Note: The eigenvector corresponding to eigenvalue 1 should be scaled to be a probability distribution
stationary_vector = eigenvectors[:, np.isclose(eigenvalues, 1)]

# Normalize the stationary vector to ensure it sums to 1
stationary_distribution = stationary_vector / np.sum(stationary_vector)

print("Stationary distribution:\n", stationary_distribution.flatten())

# Simulating and printing for 4 steps.
print("Simulating and printing for 4 steps:")
for _ in range (4):
    next_state, next_bel = weather_simulator.step(current_state, current_bel)
    print("Next state", next_state)
    print("Next belief", next_bel)
    current_state = next_state
    current_bel = next_bel



Final belief 
 [[0.64240714]
 [0.28551429]
 [0.07137857]]
Stationary distribution:
 [0.64285714 0.28571429 0.07142857]
Simulating and printing for 4 steps:
Next state 0
Next belief [[0.64240714]
 [0.28551429]
 [0.07137857]]
Next state 0
Next belief [[0.64240714]
 [0.28551429]
 [0.07137857]]
Next state 1
Next belief [[0.64240714]
 [0.28551429]
 [0.07137857]]
Next state 2
Next belief [[0.64240714]
 [0.28551429]
 [0.07137857]]


## Question 2b Write a General Purpose Simulator
Refer the implementation above.

## Question 2c
### How did you initialize the simulation?

I am initializing the simulation with a state (sunny, rainy, cloudy) and a belief for each of these states.
The belief represents what the system "thinks" of the current state.
Since the question asks to initialize from a SUNNY day, state is initialized as SUNNY.
At the beginning, I am assuming the system has a uniform belief of the state.

### How many transitions did you consider before taking a result?
The Markov Chain seems to converge to a stationary distribution after 10-20 steps. However, for a more accurate distribution, 1000 or 10000 steps are considered.

### How many simulation runs?
Running the simulation for 10, 100, 1000, 10000 transitions made the convergence of the static distributions clear.

### Closed form solution for find the static distribution

At transition t,
Xt+1 = P_transpose*Xt
Where P is the transition matrix given by the probabilities, P_transpose is its transpose,
Xt is the belief at time t and Xt+1 is the belief at time t + 1 adter applying the transition.

The belief X converges for this Markov chain after an infinite number of transition (~100 transitions practically).
Thus, the equation at convergence becomes:
X_converged = P_transpose*X_converged.

The X_converged that satisfies this equality corresponds to the Eigen Vector of P_transpose for Eigen Value 1.

Thus, a closed form solution to find the static distribution, one must needs find the eigen vector corresponding to Eigen Value 1 and normalize it to represent probabilites.
This is implemented above.

Alternatively,
First ascertain if the matrix P_transpose is diagonalizable.
A matrix A is defective if it does not have enough linearly independent eigenvectors to form a basis for its space. Specifically, for an
n×n matrix, if the number of linearly independent eigenvectors is less than n, then the matrix is defective.

Diagonalize P_transpose = A* D *A-1
The diagonal elements of D are comprised of the eigen values of P, 0,0 element being 1.

For nth transition,
Xn+1 = A * D^n * A-1 * Xn

For n tending to infinity, the diagonal elements tend to 0, leaving on the 0,0 element as 1.
This D_inf = [1, 0, 0;
                0, 0, 0;
                0, 0, 0]

X_converged = A* D_inf * A-1 which calculates to [0.64285714, 0.28571429, 0.07142857]